# Looking for features

- La fréquence d'utilisation de certains mots ou expressions clés associés à chaque parti politique.
- La longueur des phrases et la complexité du vocabulaire utilisé.
- La fréquence d'utilisation de certaines émotions ou sentiments, tels que la colère, la peur, l'espoir, etc.
- La fréquence d'utilisation de certaines structures syntaxiques ou grammaticales caractéristiques de chaque parti politique.
- La fréquence d'utilisation de certaines références culturelles ou historiques associées à chaque parti politique.
- La fréquence d'utilisation de certaines sources d'information ou de données factuelles, comme les chiffres ou les statistiques.

In [26]:
###### modules pour le chargement des données depuis le XML ######
import glob
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file

###### modules pour la classification ######

# modèles
from sklearn.svm import LinearSVC, SVC

# vectorisation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

# création de nos transformers
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer # créer nos propres transformer

# recherche des meilleurs hyperparamètres
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# résultats
from sklearn.metrics import classification_report

# sauvegarde des modèles
from joblib import dump, load

###### modules pour la visualisation ######
import matplotlib.pyplot as plt
import pandas as pd

###### miscellaneous ######
from typing import List # typage des fonctions
import numpy as np
import re
from collections import namedtuple

##### multi class #####
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

In [27]:
from nltk.corpus import stopwords

In [28]:
tree_train, root_train = get_tree_root_from_file("./corpus/train_deft09_parlement_appr.xml/deft09_parlement_appr_fr.xml")
X_train, y_train = get_X_Y_from_root(root_train)

tree_test, root_test = get_tree_root_from_file("./corpus/deft09_parlement_test.xml/deft09_parlement_test_fr.xml")
X_test, y_test = get_X_Y_from_root(root_test) # y_test est vide : pas accès aux résultats

In [29]:
pattern_clean = re.compile(r"[^ \w]") # pattern à utiliser pour nettoyer les données

def clean(data): # TODO: améliorer la fonction
    """TODO: écrire la docstring

    Args:
        data (_type_): _description_

    Returns:
        _type_: _description_
    """
    global pattern_clean
    return re.sub(pattern_clean, "", data).lower()

X_train_clean = [clean(x) for x in X_train]
X_test_clean = [clean(x) for x in X_test]


In [30]:
import re
pattern = re.compile(r'\d+\t(\w+(-\w+)?)')
y_test = []
with open("./corpus/deft09_parlement_ref/deft09_parlement_ref_fr.txt",'r') as file:
    line = file.readline()
    while line:
        m= re.match(pattern,line)
        if m:
            y_test.append(m.group(1))
        else:
            y_test.append('PSE')
        line = file.readline()

In [31]:
from sklearn.preprocessing import LabelEncoder

# Créer un objet LabelBinarizer
lb = LabelEncoder()

# Convertir les étiquettes de classe en un tableau binaire
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.fit_transform(y_test)

# Les mots les plus fréquents par partis 

In [7]:
resultat = list(zip(X_train, y_train))

In [8]:
print(y_train[:20])

['Verts-ALE', 'PPE-DE', 'Verts-ALE', 'PSE', 'PSE', 'Verts-ALE', 'PPE-DE', 'PPE-DE', 'PSE', 'PPE-DE', 'PSE', 'PPE-DE', 'ELDR', 'GUE-NGL', 'Verts-ALE', 'PSE', 'GUE-NGL', 'PSE', 'PSE', 'PSE']


In [35]:
vert = []
pse = []
ppe = []
eldr= []
gue = []

for element in resultat:
    if element[1] == "Verts-ALE":
        vert.append(element)
    if element[1] == "PPE-DE":
        ppe.append(element)
    if element[1] == "PSE":
        pse.append(element)
    if element[1] == "ELDR":
        eldr.append(element)
    if element[1] == "GUE-NGL":
            gue.append(element)
            
print(len(vert),len(pse),len(ppe),len(eldr),len(gue))

total_examples = 2376 + 5440 + 6858 + 2008 + 2688  # Nombre total d'exemples dans les données d'entraînement

weight_0 = total_examples / 2376  # Poids de la classe 0
weight_1 = total_examples / 5440  # Poids de la classe 1
weight_2 = total_examples / 6858  # Poids de la classe 2
weight_3 = total_examples / 2008  # Poids de la classe 3
weight_4 = total_examples / 2688  # Poids de la classe 4

class_weights = {0: weight_0, 1: weight_1, 2: weight_2, 3: weight_3, 4: weight_4}

2376 5440 6858 2008 2688


In [ ]:
array(['ELDR', 'GUE-NGL', 'PPE-DE', 'PSE', 'Verts-ALE'], dtype='<U9')

In [36]:
print(len(eldr),len(gue),len(ppe),len(pse),len(vert))

2008 2688 6858 5440 2376


In [37]:


total_examples = 2376 + 5440 + 6858 + 2008 + 2688  # Nombre total d'exemples dans les données d'entraînement

weight_0 = total_examples / 2008  # Poids de la classe 0
weight_1 = total_examples / 2688  # Poids de la classe 1
weight_2 = total_examples / 6858  # Poids de la classe 2
weight_3 = total_examples / 5440  # Poids de la classe 3
weight_4 = total_examples / 2378  # Poids de la classe 4

class_weights = {0: weight_0, 1: weight_1, 2: weight_2, 3: weight_3, 4: weight_4}

In [10]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

def most_frequent_words_by_label(phrases, labels):
    # Créer un dictionnaire vide qui va stocker les mots les plus fréquents pour chaque parti
    word_counts_by_label = defaultdict(lambda: defaultdict(int))
    
    # Créer une liste de stopwords en français
    stopwords_list = stopwords.words('french')
    stopwords_list.extend(["être", "avoir", "faire","’","a","commission",'monsieur', 'président','parlement', 'rapport'])
    
    # Créer un lemmatiseur
    lemmatizer = WordNetLemmatizer()

    # Pour chaque phrase et son label correspondant, compter les mots
    for phrase, label in zip(phrases, labels):
        # Tokeniser la phrase en mots
        words = word_tokenize(phrase.lower())

        # Pour chaque mot, incrémenter son compteur dans le dictionnaire de mots pour le label correspondant
        for word in words:
            # Si le mot n'est pas une stopword, n'est pas un verbe "être", "avoir" ou "faire" et n'est pas de la ponctuation, incrémenter son compteur
            if word not in stopwords_list and word not in string.punctuation:
                lemma = lemmatizer.lemmatize(word)
                if lemma not in ["être","avoir","faire"]:
                    word_counts_by_label[label][lemma] += 1

    # Pour chaque parti, trouver les 10 mots les plus fréquents
    most_frequent_words_by_label = {}
    for label, word_counts in word_counts_by_label.items():
        # Trouver les 10 mots les plus fréquents en triant le dictionnaire par compte de mot
        most_frequent_words = sorted(word_counts, key=word_counts.get, reverse=True)[:10]
        most_frequent_words_by_label[label] = most_frequent_words

    return most_frequent_words_by_label


In [11]:
phrases = [text for text,parti in resultat]
labels = [parti for text,parti in resultat]

In [12]:
print(most_frequent_words_by_label(phrases, labels))

KeyboardInterrupt: 

In [13]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


def most_frequent_words_by_label(phrases, labels):
    # Charger la liste de stop words en français
    stop_words = stopwords.words('french')
    # Ajouter la ponctuation à la liste de stop words
    stop_words += list(string.punctuation)

    # Créer un dictionnaire vide qui va stocker les mots les plus fréquents pour chaque parti
    word_counts_by_label = defaultdict(lambda: defaultdict(int))

    # Pour chaque phrase et son label correspondant, compter les mots
    for phrase, label in zip(phrases, labels):
        # Tokeniser la phrase en mots
        words = word_tokenize(phrase.lower())

        # Pour chaque mot, incrémenter son compteur dans le dictionnaire de mots pour le label correspondant
        # mais ignorer les stop words et la ponctuation
        for word in words:
            if word not in stop_words:
                word_counts_by_label[label][word] += 1

    # Pour chaque parti, trouver les 10 mots les plus fréquents
    most_frequent_words_by_label = {}
    for label, word_counts in word_counts_by_label.items():
        # Trouver les 10 mots les plus fréquents en triant le dictionnaire par compte de mot
        most_frequent_words = sorted(word_counts, key=word_counts.get, reverse=False)[:10]
        most_frequent_words_by_label[label] = most_frequent_words

    return most_frequent_words_by_label


In [14]:
print(most_frequent_words_by_label(phrases, labels))

KeyboardInterrupt: 

# Complexité du vocabulaire

In [22]:
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
# return : liste de dictionnaire où chaque dictionnaire -> complexité du vocabulaire.
def calculate_vocab_complexity(post):
    liste_finale = []
    # Créer un set pour stocker les mots uniques
    unique_words = set()

    # Pour chaque phrase, ajouter les mots uniques au set
    for phrases in post:
        vocab_complexity = {}
        for phrase in phrases.split():
            words = word_tokenize(phrase)
            for word in words:
                unique_words.add(word)

        # Compter le nombre total de mots
        total_word_count = 0
        for phrase in phrases:
            total_word_count += len(word_tokenize(phrase))

        # Calculer la complexité du vocabulaire en divisant le nombre de mots uniques par le nombre total de mots
        vocab_complexity["complexity"] = len(unique_words) / total_word_count
        liste_finale.append(vocab_complexity)
    return liste_finale


In [23]:
calculate_vocab_complexity(["Julie est là","Julie est aussi ici"])

[{'complexity': 0.3}, {'complexity': 0.3125}]

In [24]:
# Ajoutez les prédictions de thèmes à votre pipeline de classification de discours politiques
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
pipeline_avec_complex = Pipeline([
    ('features', FeatureUnion([
        ('vec', Pipeline([
            ('count', CountVectorizer(min_df=2, max_df=0.8)),
            ('tf', TfidfTransformer())
        ])),
        ('theme', Pipeline([
          ('stats', FunctionTransformer(calculate_vocab_complexity)),
          ('vect', DictVectorizer())
        ]))
    ])),
    #('pca', PCA()),  # Ajout de la réduction de la dimensionalité par ACP ici
    ('standard', StandardScaler(with_mean=False)),
    ('rForest', RandomForestClassifier(criterion='gini',max_features='sqrt'))
])

# Entraînez le modèle de classification de discours politiques sur les données d'entraînement
#pipeline_avec_theme.fit(X_train_sample, y_train_sample)

cv_avec_complex = cross_val_score(pipeline_avec_complex, X_train_clean[:500], y_train_bin[:500], verbose=10, cv=3)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] START .....................................................................
[CV] END ................................ score: (test=0.335) total time= 1.1min
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV] END ................................ score: (test=0.335) total time= 1.3min
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.4min remaining:    0.0s


[CV] END ................................ score: (test=0.349) total time= 1.4min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.8min finished


In [25]:
pipeline_avec_complex.fit(X_train_clean[:100],y_train_bin[:100])
y_pred = pipeline_avec_complex.predict(X_test_clean[:500])
print(classification_report(y_test_bin[:500],y_pred))

              precision    recall  f1-score   support

           0       1.00      0.02      0.04        52
           1       0.62      0.07      0.12        73
           2       0.39      0.39      0.39       177
           3       0.27      0.62      0.38       128
           4       0.16      0.04      0.07        70

    accuracy                           0.31       500
   macro avg       0.49      0.23      0.20       500
weighted avg       0.42      0.31      0.26       500



# Les émotions

Il existe plusieurs lexiques de sentiments pour le français. Vous pouvez par exemple utiliser le lexique "LIWC" (Linguistic Inquiry and Word Count), qui contient plus de 6 000 mots et expressions en français associés à des émotions et sentiments. Vous pouvez également utiliser le lexique "AFINN", qui contient environ 2 500 mots et expressions en français associés à des émotions et sentiments, ainsi que des scores de polarité positive et négative. Vous pouvez également utiliser le lexique "NRC Emotion Lexicon", qui contient environ 14 000 mots et expressions en français associés à huit émotions différentes (joie, peur, colère, tristesse, surprise, dégoût, confiance et anticipation).

# Les structures grammaticales et syntaxiques

In [97]:
from nltk import ngrams

def count_4grams(phrases):
    for phrase in phrases:
        # Tokeniser la phrase en mots
        words = word_tokenize(phrase)

        # Générer les 4-grams de la phrase
        four_grams = ngrams(words, 4)

        # Compter les 4-grams
        four_gram_counts = defaultdict(int)
        for four_gram in four_grams:
            four_gram_counts[four_gram] += 1
    
    return sorted(four_gram_counts, key=four_gram_counts.get, reverse=True)[:10]


In [99]:
count_4grams(phrases)

[('Monsieur', 'le', 'Président', ','),
 ('le', 'Président', ',', 'le'),
 ('Président', ',', 'le', 'rapport'),
 (',', 'le', 'rapport', 'de'),
 ('le', 'rapport', 'de', 'Mme'),
 ('rapport', 'de', 'Mme', 'Zorba'),
 ('de', 'Mme', 'Zorba', 'est'),
 ('Mme', 'Zorba', 'est', 'très'),
 ('Zorba', 'est', 'très', 'intelligent'),
 ('est', 'très', 'intelligent', ',')]

In [102]:
from collections import defaultdict
from nltk import pos_tag, word_tokenize

def most_frequent_pos_4grams(phrase):
    # Tokeniser la phrase en mots
    for phrase in phrases:
        words = word_tokenize(phrase)

        # Tagger les mots avec leur partie du discours
        pos_tags = pos_tag(words)

        # Créer un dictionnaire vide qui va stocker les 4-grams de POS les plus fréquents
        pos_4gram_counts = defaultdict(int)

        # Pour chaque 4-gram de POS, incrémenter son compteur dans le dictionnaire
        for i in range(len(pos_tags) - 3):
            pos_4gram = pos_tags[i][1] + " " + pos_tags[i+1][1] + " " + pos_tags[i+2][1] + " " + pos_tags[i+3][1]
            pos_4gram_counts[pos_4gram] += 1

        # Trouver les 10 4-grams de POS les plus fréquents en triant le dictionnaire par compte de 4-gram
        most_frequent_pos_4grams = sorted(pos_4gram_counts, key=pos_4gram_counts.get, reverse=True)[:10]

    return most_frequent_pos_4grams

In [103]:
most_frequent_pos_4grams(phrases[:100])

['JJ NN NN NN',
 'NN NN JJ NN',
 'NN JJ NN NN',
 'JJ NN IN NN',
 'FW FW NN FW',
 'NN NN NN NN',
 'FW FW FW NN',
 'NNS VBP JJ NN',
 'NN NN , JJ',
 'JJ NN NN JJ']